# Import Libraries

In [2]:
#Importing General Library
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import missingno as msno 

#Sklearn Library
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, f1_score, recall_score, accuracy_score, precision_score
from imblearn.over_sampling import RandomOverSampler
from sklearn import preprocessing

#Keras Library
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#NLTK Library
import nltk
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn.metrics import plot_confusion_matrix, classification_report, f1_score, recall_score
from sklearn.metrics import precision_recall_curve, average_precision_score

#Import yellowbrick Library
#pip install -U yellowbrick ######
from yellowbrick.classifier import PrecisionRecallCurve

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# Reading the Data


# Machine Learning

In [4]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [16]:
df = pd.read_excel("/content/drive/MyDrive/DeGatto Project/EDA + Models/Aspect Annotated Data.xlsx")

In [17]:
df = df.drop('Unnamed: 0', axis = 1)

In [9]:
x = df["ReviewText"]
size = df["Size"]
material = df['Material']
design = df['Design']
comfort = df['Comfort']

**Count Vectorizer**

In [10]:
def vectorization(x_train, x_test):
  vectorizer = CountVectorizer()
  x_train_count = vectorizer.fit_transform(x_train)
  x_test_count = vectorizer.transform(x_test)
  return x_train_count, x_test_count

**TF-IDF**

In [11]:
def tf_idf_transformation(x_train, x_test):
  tf_idf_vectorizer = TfidfVectorizer()
  x_train_tf_idf = tf_idf_vectorizer.fit_transform(x_train)
  x_test_tf_idf = tf_idf_vectorizer.transform(x_test)
  return x_train_tf_idf, x_test_tf_idf

In [18]:
df_rest = df.drop(['ReviewText'], axis = 1).replace(['pos', 'no', 'neg', 'neu'], [3, 0, 1, 2])

df.drop(['Size', 'Material', 'Design', 'Comfort'], axis = 1, inplace = True)
df = df.join(df_rest)

In [19]:
df

,ReviewText,Size,Material,Design,Comfort
0,i am in need of easy comfortable tops for ever...,1,0,0,3
1,i read the previous reviews and had hoped that...,0,3,0,3
2,this is exactly what i was expecting cute comf...,0,0,3,3
3,this dress is gorgeous i love it i bought it t...,3,0,3,3
4,love this top made with 100 cotton a vintage l...,3,0,0,3
...,...,...,...,...,...
2886,i was very happy to snag this dress at such a ...,0,3,3,3
2887,it reminds me of maternity clothes soft stretc...,0,3,0,0
2888,this fit well but the top was very see through...,3,1,0,0
2889,i bought this dress for a wedding i have this ...,1,0,0,0


**Logistic Regression**

In [26]:
test_size = []

accuracy_test = []
accuracy_train = []

recall_train = []
recall_test = []

precision_train = []
precision_test = []

f1score_train = []
f1score_test = []

vector_method = []
c_value = []
max_iterations = []

vectorizer = CountVectorizer()
tf_idf_vectorizer = TfidfVectorizer()
over = RandomOverSampler(sampling_strategy = 'all', random_state = 45)

output_to_predict = []

out_preds = {"Size": df["Size"], "Material": df["Material"],
            "Design": df["Design"], "Comfort":df["Comfort"]}

for c_name, out_pred in out_preds.items():
  for i in range(2):
    for sizes in [0.1, 0.15, 0.2,0.25,0.3]:
      for c_rv in [0.01, 0.05, 0.1]: 
        for iterations_max in range(100, 500, 100): 
          output_to_predict.append(c_name)
          max_iterations.append(iterations_max)
          c_value.append(c_rv)
          test_size.append(sizes)

          x = df["ReviewText"]
          y = out_pred

          if(i == 0):
            x = vectorizer.fit_transform(x)
            ## Stratify sampling is used in NLP to remove the bias
            x, y = over.fit_resample(x, y)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = sizes, stratify = y, random_state = 42)

            vector_method.append("Count Vectorizer")

          else:
            x = tf_idf_vectorizer.fit_transform(x)
            x, y = over.fit_resample(x, y)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = sizes, stratify = y, random_state = 42)

            vector_method.append("TF-IDF")

          log = LogisticRegression(C = c_rv, max_iter=iterations_max, class_weight= "balanced", random_state=101)
          log.fit(x_train,y_train)
          
          y_pred_test = log.predict(x_test)
          y_pred_train = log.predict(x_train)
          
          accuracy_test.append(accuracy_score(y_test, y_pred_test))
          accuracy_train.append(accuracy_score(y_train, y_pred_train))

          recall_train.append(recall_score(y_train, y_pred_train, average = 'weighted'))
          recall_test.append(recall_score(y_test, y_pred_test, average = 'weighted'))

          precision_train.append(precision_score(y_train, y_pred_train, average = 'weighted'))
          precision_test.append(precision_score(y_test, y_pred_test, average = 'weighted'))

          f1score_train.append(f1_score(y_train, y_pred_train, average = 'weighted'))
          f1score_test.append(f1_score(y_test, y_pred_test, average = 'weighted'))
    
dict = {"Output Predicted":  output_to_predict, 'Vectorization Method': vector_method,  'Test Size': test_size, "C value": c_value, 'Max Iterations': max_iterations,
        'Accuracy Train':accuracy_train, 'Accuracy Test': accuracy_test, 'Precision Train': precision_train, 'Precision Test': precision_test,
        'Recall Train': recall_train, 'Recall Test': recall_test, 'F1 Score Train': f1score_train, 'F1 Score Test': f1score_test}
log_df_1 = pd.DataFrame(dict)

In [27]:
log_df_1

,Output Predicted,Vectorization Method,Test Size,C value,Max Iterations,Accuracy Train,Accuracy Test,Precision Train,Precision Test,Recall Train,Recall Test,F1 Score Train,F1 Score Test
0,Size,Count Vectorizer,0.1,0.01,100,0.762329,0.721698,0.765090,0.722540,0.762329,0.721698,0.761147,0.719597
1,Size,Count Vectorizer,0.1,0.01,200,0.762329,0.721698,0.765090,0.722540,0.762329,0.721698,0.761147,0.719597
2,Size,Count Vectorizer,0.1,0.01,300,0.762329,0.721698,0.765090,0.722540,0.762329,0.721698,0.761147,0.719597
3,Size,Count Vectorizer,0.1,0.01,400,0.762329,0.721698,0.765090,0.722540,0.762329,0.721698,0.761147,0.719597
4,Size,Count Vectorizer,0.1,0.05,100,0.884313,0.783019,0.885117,0.784778,0.884313,0.783019,0.884169,0.781656
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,Comfort,TF-IDF,0.3,0.05,400,0.931404,0.917747,0.931073,0.917677,0.931404,0.917747,0.930946,0.917135
476,Comfort,TF-IDF,0.3,0.10,100,0.949224,0.935181,0.949485,0.935936,0.949224,0.935181,0.948966,0.934757
477,Comfort,TF-IDF,0.3,0.10,200,0.949224,0.935181,0.949485,0.935936,0.949224,0.935181,0.948966,0.934757
478,Comfort,TF-IDF,0.3,0.10,300,0.949224,0.935181,0.949485,0.935936,0.949224,0.935181,0.948966,0.934757


**Multinomial Naive Bayes**

In [22]:
test_size = []

accuracy_test = []
accuracy_train = []

recall_train = []
recall_test = []

precision_train = []
precision_test = []

f1score_train = []
f1score_test = []

vector_method = []
alpha_value = []

vectorizer = CountVectorizer()
tf_idf_vectorizer = TfidfVectorizer()
over = RandomOverSampler(sampling_strategy = 'all')

output_to_predict = []

out_preds = {"Size": df["Size"], "Material": df["Material"],
            "Design": df["Design"], "Comfort":df["Comfort"]}

for c_name, out_pred in out_preds.items():
  for i in range(2):
    for sizes in [0.1, 0.15, 0.2,0.25,0.3]:
      for alp_ha in [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:   
        output_to_predict.append(c_name)
        alpha_value.append(alp_ha)
        test_size.append(sizes)

        x = df["ReviewText"]
        y = out_pred

        if(i == 0):
          x = vectorizer.fit_transform(x)
          ## Stratify sampling is used in NLP to remove the bias
          x, y = over.fit_resample(x, y)
          x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = sizes, stratify = y, random_state = 42)

          vector_method.append("Count Vectorizer")

        else:
          x = tf_idf_vectorizer.fit_transform(x)
          x, y = over.fit_resample(x, y)
          x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = sizes, stratify = y, random_state = 42)

          vector_method.append("TF-IDF")

        mnb = MultinomialNB(alpha = alp_ha)
        mnb.fit(x_train, y_train)
        
        y_pred_test = mnb.predict(x_test)
        y_pred_train = mnb.predict(x_train)
        
        accuracy_test.append(accuracy_score(y_test, y_pred_test))
        accuracy_train.append(accuracy_score(y_train, y_pred_train))

        recall_train.append(recall_score(y_train, y_pred_train, average = 'weighted'))
        recall_test.append(recall_score(y_test, y_pred_test, average = 'weighted'))

        precision_train.append(precision_score(y_train, y_pred_train, average = 'weighted'))
        precision_test.append(precision_score(y_test, y_pred_test, average = 'weighted'))

        f1score_train.append(f1_score(y_train, y_pred_train, average = 'weighted'))
        f1score_test.append(f1_score(y_test, y_pred_test, average = 'weighted'))
    
dict = {"Output Predicted":  output_to_predict, 'Vectorization Method': vector_method, 
        'Test Size': test_size, 'Alpha': alpha_value, 'Accuracy Train':accuracy_train, 'Accuracy Test': accuracy_test, 
        'Precision Train': precision_train, 'Precision Test': precision_test,
        'Recall Train': recall_train, 'Recall Test': recall_test, 
        'F1 Score Train': f1score_train, 'F1 Score Test': f1score_test}
MNB_df = pd.DataFrame(dict)

In [23]:
dict = {"Output Predicted":  output_to_predict, 'Vectorization Method': vector_method, 
        'Test Size': test_size, 'Alpha': alpha_value, 'Accuracy Train':accuracy_train, 'Accuracy Test': accuracy_test, 
        'Precision Train': precision_train, 'Precision Test': precision_test,
        'Recall Train': recall_train, 'Recall Test': recall_test, 
        'F1 Score Train': f1score_train, 'F1 Score Test': f1score_test}
MNB_df = pd.DataFrame(dict)
MNB_df

,Output Predicted,Vectorization Method,Test Size,Alpha,Accuracy Train,Accuracy Test,Precision Train,Precision Test,Recall Train,Recall Test,F1 Score Train,F1 Score Test
0,Size,Count Vectorizer,0.1,0.0,0.932844,0.757075,0.933029,0.755299,0.932844,0.757075,0.932884,0.755772
1,Size,Count Vectorizer,0.1,0.5,0.885362,0.768868,0.886249,0.770044,0.885362,0.768868,0.885454,0.768686
2,Size,Count Vectorizer,0.1,1.0,0.871196,0.757075,0.874222,0.759971,0.871196,0.757075,0.871525,0.757678
3,Size,Count Vectorizer,0.1,1.5,0.848636,0.745283,0.853079,0.751367,0.848636,0.745283,0.848945,0.744938
4,Size,Count Vectorizer,0.1,2.0,0.838930,0.724057,0.845354,0.739050,0.838930,0.724057,0.839250,0.726676
...,...,...,...,...,...,...,...,...,...,...,...,...
275,Comfort,TF-IDF,0.3,1.0,0.940985,0.895396,0.941159,0.894286,0.940985,0.895396,0.940494,0.894503
276,Comfort,TF-IDF,0.3,1.5,0.941176,0.895396,0.941312,0.894425,0.941176,0.895396,0.940636,0.893914
277,Comfort,TF-IDF,0.3,2.0,0.933512,0.886455,0.933170,0.884782,0.933512,0.886455,0.932773,0.884777
278,Comfort,TF-IDF,0.3,2.5,0.931404,0.883773,0.930666,0.880912,0.931404,0.883773,0.930504,0.881909


**SVM**

In [24]:
test_size = []

accuracy_test = []
accuracy_train = []

recall_train = []
recall_test = []

precision_train = []
precision_test = []

f1score_train = []
f1score_test = []

vector_method = []
maxiter = []
c_value = []

vectorizer = CountVectorizer()
tf_idf_vectorizer = TfidfVectorizer()
over = RandomOverSampler(sampling_strategy = 'all')

output_to_predict = []

out_preds = {"Size": df["Size"], "Material": df["Material"],
            "Design": df["Design"], "Comfort":df["Comfort"]}

for c_name, out_pred in out_preds.items():
  for i in range(2):
    for sizes in [0.1, 0.15, 0.2,0.25,0.3]:
      for c_rv in [0.01, 0.03, 0.05, 0.07, 0.1]:  
        for mi in [50,100,150,200,250,300]:
          output_to_predict.append(c_name)
          c_value.append(c_rv)
          test_size.append(sizes)
          maxiter.append(mi)

          x = df["ReviewText"]
          y = out_pred

          if(i == 0):
            x = vectorizer.fit_transform(x)
            ## Stratify sampling is used in NLP to remove the bias
            x, y = over.fit_resample(x, y)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = sizes, stratify = y, random_state = 42)

            vector_method.append("Count Vectorizer")

          else:
            x = tf_idf_vectorizer.fit_transform(x)
            x, y = over.fit_resample(x, y)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = sizes, stratify = y, random_state = 42)

            vector_method.append("TF-IDF")

          svc = LinearSVC(C=c_rv, class_weight="balanced", max_iter = mi, random_state=101)
          svc.fit(x_train,y_train)
          
          y_pred_test = svc.predict(x_test)
          y_pred_train = svc.predict(x_train)
          
          accuracy_test.append(accuracy_score(y_test, y_pred_test))
          accuracy_train.append(accuracy_score(y_train, y_pred_train))

          recall_train.append(recall_score(y_train, y_pred_train, average = 'weighted'))
          recall_test.append(recall_score(y_test, y_pred_test, average = 'weighted'))

          precision_train.append(precision_score(y_train, y_pred_train, average = 'weighted'))
          precision_test.append(precision_score(y_test, y_pred_test, average = 'weighted'))

          f1score_train.append(f1_score(y_train, y_pred_train, average = 'weighted'))
          f1score_test.append(f1_score(y_test, y_pred_test, average = 'weighted'))
      
dict = {"Output Predicted":  output_to_predict, 'Vectorization Method': vector_method, 
        'Test Size': test_size, 'C Value': c_value, 'Maximum iterations': maxiter, 
        'Accuracy Train':accuracy_train, 'Accuracy Test': accuracy_test, 'Precision Train': precision_train, 'Precision Test': precision_test,
        'Recall Train': recall_train, 'Recall Test': recall_test, 'F1 Score Train': f1score_train, 'F1 Score Test': f1score_test}
svm_df_1 = pd.DataFrame(dict)

In [25]:
svm_df_1

,Output Predicted,Vectorization Method,Test Size,C Value,Maximum iterations,Accuracy Train,Accuracy Test,Precision Train,Precision Test,Recall Train,Recall Test,F1 Score Train,F1 Score Test
0,Size,Count Vectorizer,0.1,0.01,50,0.905824,0.801887,0.906462,0.802698,0.905824,0.801887,0.905682,0.799917
1,Size,Count Vectorizer,0.1,0.01,100,0.905037,0.806604,0.905361,0.808352,0.905037,0.806604,0.904775,0.805618
2,Size,Count Vectorizer,0.1,0.01,150,0.907922,0.778302,0.908353,0.778111,0.907922,0.778302,0.907789,0.776292
3,Size,Count Vectorizer,0.1,0.01,200,0.905037,0.759434,0.905664,0.764600,0.905037,0.759434,0.904875,0.758302
4,Size,Count Vectorizer,0.1,0.01,250,0.903987,0.785377,0.904737,0.785172,0.903987,0.785377,0.903938,0.783783
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Comfort,TF-IDF,0.3,0.10,100,0.982564,0.968708,0.982748,0.969463,0.982564,0.968708,0.982549,0.968623
1196,Comfort,TF-IDF,0.3,0.10,150,0.982372,0.962450,0.982551,0.963743,0.982372,0.962450,0.982354,0.962318
1197,Comfort,TF-IDF,0.3,0.10,200,0.980839,0.966920,0.981122,0.967587,0.980839,0.966920,0.980823,0.966827
1198,Comfort,TF-IDF,0.3,0.10,250,0.982947,0.962450,0.983053,0.963771,0.982947,0.962450,0.982935,0.962298


**LSTM**

**Saving In Excel**

In [29]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 KB 7.4 MB/s eta 0:00:00


In [ ]:
with pd.ExcelWriter('/content/drive/MyDrive/DeGatto Project/EDA + Models/First Output 3.xlsx', engine='xlsxwriter') as writer:
  svm_df_1.to_excel(writer, sheet_name='SVM')

In [30]:
with pd.ExcelWriter('/content/drive/MyDrive/DeGatto Project/EDA + Models/First Output 1.xlsx', engine='xlsxwriter') as writer:
    log_df_1.to_excel(writer, sheet_name='Logistic Regression')
    MNB_df.to_excel(writer, sheet_name='Multinomial Naive Bayes')
    svm_df_1.to_excel(writer, sheet_name='SVM')